In [1]:
import os 
import random 
import logging 

import torch 
import mlflow 
import numpy as np 
import matplotlib.pyplot as plt 
import torchvision.transforms as transforms
import torchvision.utils 
import torch.nn as nn 

%matplotlib inline

In [3]:
os.environ['USER'] = 'Denis Rusinov'

In [4]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [6]:
from flask.cli import F


train_mnist = torchvision.datasets.MNIST('data_mnist/train', train=True, 
                                         transform=transforms.Compose([transforms.ToTensor()]),
                                         download=True)
val_mnist = torchvision.datasets.MNIST('data_mnist/test', train=False, 
                                       transform=transforms.Compose([transforms.ToTensor()]),
                                       download=True)

batch_size = 256

train_loader = torch.utils.data.DataLoader(
    dataset=train_mnist,
    batch_size=batch_size,
    shuffle=True
)

val_loader = torch.utils.data.DataLoader(
    dataset=val_mnist,
    batch_size=batch_size,
    shuffle=False
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data_mnist/train\MNIST\raw\train-images-idx3-ubyte.gz to data_mnist/train\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data_mnist/train\MNIST\raw\train-labels-idx1-ubyte.gz to data_mnist/train\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



...

Extracting data_mnist/test\MNIST\raw\train-labels-idx1-ubyte.gz to data_mnist/test\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data_mnist/test\MNIST\raw\t10k-images-idx3-ubyte.gz to data_mnist/test\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting data_mnist/test\MNIST\raw\t10k-labels-idx1-ubyte.gz to data_mnist/test\MNIST\raw



In [7]:
class FCNetwork(nn.Module):
    def __init__(self, prob, n_inside):
        super(FCNetwork, self).__init__() 
        self.fc1 = nn.Linear(784, n_inside)
        self.fc2 = nn.Linear(n_inside, 10)
        self.fc1_act = nn.ReLU()
        self.dropout = nn.Dropout(p = prob)

    def forward(self,x):
        x = x.view(-1,28*28)
        y = self.fc1(self.dropout(x))
        y = self.fc1_act(y)
        y = self.fc2(y)
        return y

In [8]:
# Инициализация MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("PyTorch_test")

2024/12/16 16:06:17 INFO mlflow.tracking.fluent: Experiment with name 'PyTorch_test' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/Rusinov.DS/PycharmProjects/ML_project/dns_purchase/artefacts/980200151464093923', creation_time=1734329177469, experiment_id='980200151464093923', last_update_time=1734329177469, lifecycle_stage='active', name='PyTorch_test', tags={}>

In [9]:
# Отключаем вывод ворнингов от MLflow
mlflow_logger = logging.getLogger("mlflow")
mlflow_logger.setLevel(logging.ERROR)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
prob = 0.15
n_inside = 50
lr = 1e-3
epochs = 15

In [16]:
# Функция для вычисления точности
def accuracy(y_pred, labels):
    preds = torch.argmax(y_pred, dim=1)
    return (preds == labels).float().mean().item()

In [17]:
# Начало MLflow запуска
with mlflow.start_run(run_name='FCNetwork_1') as run:
    model = FCNetwork(prob=prob, n_inside=n_inside)
    model.to(device)

    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    mlflow.log_param("prob dropout", prob)
    mlflow.log_param("neurons 2 layer", n_inside)
    mlflow.log_param("lr", lr)
    mlflow.log_param("optimizer", 'Adam')
    mlflow.log_param("epochs", epochs)

    maxacc = 0
    itr_record = 0

    for epoch in range(epochs):
        epoch += 1
        model.train()
        train_loss = 0.0
        train_acc = 0.0
        train_samples = 0

        print(f'Началось обучение {epoch} эпохи')
        for itr, data in enumerate(train_loader):
            imgs = data[0].to(device)  # [B, H, W]
            labels = data[1].to(device)

            y_pred = model(imgs) 
            loss = loss_func(y_pred, labels)

            train_loss += loss.item() * imgs.size(0)
            train_acc += accuracy(y_pred, labels) * imgs.size(0)
            train_samples += imgs.size(0)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_loss /= train_samples
        train_acc /= train_samples
        mlflow.log_metric("train_loss", train_loss, step=epoch)
        mlflow.log_metric("train_acc", train_acc, step=epoch)
        print(f'The {epoch} Epoch of network learning is over:')
        print(f'Train results Epoch {epoch}: Train loss - {train_loss:.4f}, Train accuracy - {train_acc:.4f}')

        model.eval()
        val_loss = 0.0
        val_acc = 0.0
        val_samples = 0

        with torch.no_grad():
            for itr, data in enumerate(val_loader):
                imgs = data[0].to(device)
                labels = data[1].to(device)
                y_pred = model(imgs)
                loss = loss_func(y_pred, labels)

                val_loss += loss.item() * imgs.size(0)
                val_acc += accuracy(y_pred, labels) * imgs.size(0)
                val_samples += imgs.size(0)

        val_loss /= val_samples
        val_acc /= val_samples
        mlflow.log_metric("val_loss", val_loss, step=epoch)
        mlflow.log_metric("val_acc", val_acc, step=epoch)
        print(f'Validation results Epoch {epoch}: Val loss - {val_loss:.4f}, Test accuracy - {val_acc:.4f}')

        if val_acc > maxacc:
            print('Saving model because its better')
            maxacc = val_acc
            mlflow.pytorch.log_model(model, "model")
        print('---')

    print('max accuracy = ', maxacc)
    mlflow.log_metric("max val accuracy", maxacc)

mlflow.end_run()

Началось обучение 1 эпохи
The 1 Epoch of network learning is over:
Train results Epoch 1: Train loss - 0.6602, Train accuracy - 0.8283
Validation results Epoch 1: Val loss - 0.3185, Test accuracy - 0.9098
Saving model because its better
---
Началось обучение 2 эпохи
The 2 Epoch of network learning is over:
Train results Epoch 2: Train loss - 0.3177, Train accuracy - 0.9091
Validation results Epoch 2: Val loss - 0.2610, Test accuracy - 0.9266
Saving model because its better
---
Началось обучение 3 эпохи
The 3 Epoch of network learning is over:
Train results Epoch 3: Train loss - 0.2723, Train accuracy - 0.9224
Validation results Epoch 3: Val loss - 0.2274, Test accuracy - 0.9357
Saving model because its better
---
Началось обучение 4 эпохи
The 4 Epoch of network learning is over:
Train results Epoch 4: Train loss - 0.2362, Train accuracy - 0.9324
Validation results Epoch 4: Val loss - 0.1945, Test accuracy - 0.9444
Saving model because its better
---
Началось обучение 5 эпохи
The 5 Epoc